In [1]:
from lionagi.experimental.compressor.llm_compressor import LLMCompressor

In [2]:
aa = """
Introduction
You can interact with the API through HTTP requests from any language, via our official Python bindings, our official Node.js library, or a community-maintained library.

To install the official Python bindings, run the following command:

pip install openai
To install the official Node.js library, run the following command in your Node.js project directory:

npm install openai@^4.0.0
Authentication
API keys
The OpenAI API uses API keys for authentication. You can create API keys at a user or service account level. Service accounts are tied to a "bot" individual and should be used to provision access for production systems. Each API key can be scoped to one of the following,

Project keys - Provides access to a single project (preferred option); access Project API keys by selecting the specific project you wish to generate keys against.
User keys - Our legacy keys. Provides access to all organizations and all projects that user has been added to; access API Keys to view your available keys. We highly advise transitioning to project keys for best security practices, although access via this method is currently still supported.
Remember that your API key is a secret! Do not share it with others or expose it in any client-side code (browsers, apps). Production requests must be routed through your own backend server where your API key can be securely loaded from an environment variable or key management service.

All API requests should include your API key in an Authorization HTTP header as follows:

Authorization: Bearer OPENAI_API_KEY
Organizations and projects (optional)
For users who belong to multiple organizations or are accessing their projects through their legacy user API key, you can pass a header to specify which organization and project is used for an API request. Usage from these API requests will count as usage for the specified organization and project.

To access the Default project in an organization, leave out the OpenAI-Project header

Example curl command:

curl https://api.openai.com/v1/models \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -H "OpenAI-Organization: org-rU9EfyXF6LGImfhnZE2D0VrJ" \
  -H "OpenAI-Project: $PROJECT_ID"
Example with the openai Python package:

from openai import OpenAI

client = OpenAI(
  organization='org-rU9EfyXF6LGImfhnZE2D0VrJ',
  project='$PROJECT_ID',
)
Example with the openai Node.js package:

import OpenAI from "openai";

const openai = new OpenAI({
    organization: "org-rU9EfyXF6LGImfhnZE2D0VrJ",
    project: "$PROJECT_ID",
});
Organization IDs can be found on your Organization settings page. Project IDs can be found on your General settings page by selecting the specific project.

Making requests
You can paste the command below into your terminal to run your first API request. Make sure to replace $OPENAI_API_KEY with your secret API key. If you are using a legacy user key and you have multiple projects, you will also need to specify the Project Id. For improved security, we recommend transitioning to project based keys instead.

curl https://api.openai.com/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
     "model": "gpt-3.5-turbo",
     "messages": [{"role": "user", "content": "Say this is a test!"}],
     "temperature": 0.7
   }'
This request queries the gpt-3.5-turbo model (which under the hood points to a gpt-3.5-turbo model variant) to complete the text starting with a prompt of "Say this is a test". You should get a response back that resembles the following:

{
    "id": "chatcmpl-abc123",
    "object": "chat.completion",
    "created": 1677858242,
    "model": "gpt-3.5-turbo-0613",
    "usage": {
        "prompt_tokens": 13,
        "completion_tokens": 7,
        "total_tokens": 20
    },
    "choices": [
        {
            "message": {
                "role": "assistant",
                "content": "\n\nThis is a test!"
            },
            "logprobs": null,
            "finish_reason": "stop",
            "index": 0
        }
    ]
}
Now that you've generated your first chat completion, let's break down the response object. We can see the finish_reason is stop which means the API returned the full chat completion generated by the model without running into any limits. In the choices list, we only generated a single message but you can set the n parameter to generate multiple messages choices.

Streaming
The OpenAI API provides the ability to stream responses back to a client in order to allow partial results for certain requests. To achieve this, we follow the Server-sent events standard. Our official Node and Python libraries include helpers to make parsing these events simpler.

Streaming is supported for both the Chat Completions API and the Assistants API. This section focuses on how streaming works for Chat Completions. Learn more about how streaming works in the Assistants API here.

In Python, a streaming request looks like:

from openai import OpenAI

client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
In Node / Typescript, a streaming request looks like:

import OpenAI from "openai";

const openai = new OpenAI();

async function main() {
    const stream = await openai.chat.completions.create({
        model: "gpt-3.5-turbo",
        messages: [{ role: "user", content: "Say this is a test" }],
        stream: true,
    });
    for await (const chunk of stream) {
        process.stdout.write(chunk.choices[0]?.delta?.content || "");
    }
}

main();
Parsing Server-sent events
Parsing Server-sent events is non-trivial and should be done with caution. Simple strategies like splitting by a new line may result in parsing errors. We recommend using existing client libraries when possible.

Audio
Learn how to turn audio into text or text into audio.

Related guide: Speech to text

Create speech
POST
 
https://api.openai.com/v1/audio/speech

Generates audio from the input text.

Request body
model
string

Required
One of the available TTS models: tts-1 or tts-1-hd

input
string

Required
The text to generate audio for. The maximum length is 4096 characters.

voice
string

Required
The voice to use when generating the audio. Supported voices are alloy, echo, fable, onyx, nova, and shimmer. Previews of the voices are available in the Text to speech guide.

response_format
string

Optional
Defaults to mp3
The format to audio in. Supported formats are mp3, opus, aac, flac, wav, and pcm.

speed
number

Optional
Defaults to 1
The speed of the generated audio. Select a value from 0.25 to 4.0. 1.0 is the default.

Returns
The audio file content.

Example request
curl

curl
curl https://api.openai.com/v1/audio/speech \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -H "Content-Type: application/json" \
  -d '{
    "model": "tts-1",
    "input": "The quick brown fox jumped over the lazy dog.",
    "voice": "alloy"
  }' \
  --output speech.mp3
Create transcription
POST
 
https://api.openai.com/v1/audio/transcriptions

Transcribes audio into the input language.

Request body
file
file

Required
The audio file object (not file name) to transcribe, in one of these formats: flac, mp3, mp4, mpeg, mpga, m4a, ogg, wav, or webm.

model
string

Required
ID of the model to use. Only whisper-1 (which is powered by our open source Whisper V2 model) is currently available.

language
string

Optional
The language of the input audio. Supplying the input language in ISO-639-1 format will improve accuracy and latency.

prompt
string

Optional
An optional text to guide the model's style or continue a previous audio segment. The prompt should match the audio language.

response_format
string

Optional
Defaults to json
The format of the transcript output, in one of these options: json, text, srt, verbose_json, or vtt.

temperature
number

Optional
Defaults to 0
The sampling temperature, between 0 and 1. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic. If set to 0, the model will use log probability to automatically increase the temperature until certain thresholds are hit.

timestamp_granularities[]
array

Optional
Defaults to segment
The timestamp granularities to populate for this transcription. response_format must be set verbose_json to use timestamp granularities. Either or both of these options are supported: word, or segment. Note: There is no additional latency for segment timestamps, but generating word timestamps incurs additional latency.

Returns
The transcription object or a verbose transcription object.

Example request
curl

curl
curl https://api.openai.com/v1/audio/transcriptions \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -H "Content-Type: multipart/form-data" \
  -F file="@/path/to/file/audio.mp3" \
  -F model="whisper-1"
Response
{
  "text": "Imagine the wildest idea that you've ever had, and you're curious about how it might scale to something that's a 100, a 1,000 times bigger. This is a place where you can get to do that."
}
Create translation
POST
 
https://api.openai.com/v1/audio/translations

Translates audio into English.

Request body
file
file

Required
The audio file object (not file name) translate, in one of these formats: flac, mp3, mp4, mpeg, mpga, m4a, ogg, wav, or webm.

model
string

Required
ID of the model to use. Only whisper-1 (which is powered by our open source Whisper V2 model) is currently available.

prompt
string

Optional
An optional text to guide the model's style or continue a previous audio segment. The prompt should be in English.

response_format
string

Optional
Defaults to json
The format of the transcript output, in one of these options: json, text, srt, verbose_json, or vtt.

temperature
number

Optional
Defaults to 0
The sampling temperature, between 0 and 1. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic. If set to 0, the model will use log probability to automatically increase the temperature until certain thresholds are hit.

Returns
The translated text.

Example request
curl

curl
curl https://api.openai.com/v1/audio/translations \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -H "Content-Type: multipart/form-data" \
  -F file="@/path/to/file/german.m4a" \
  -F model="whisper-1"
Response
{
  "text": "Hello, my name is Wolfgang and I come from Germany. Where are you heading today?"
}
The transcription object (JSON)
Represents a transcription response returned by model, based on the provided input.

text
string

The transcribed text.

The transcription object (JSON)
{
  "text": "Imagine the wildest idea that you've ever had, and you're curious about how it might scale to something that's a 100, a 1,000 times bigger. This is a place where you can get to do that."
}
The transcription object (Verbose JSON)
Represents a verbose json transcription response returned by model, based on the provided input.

language
string

The language of the input audio.

duration
string

The duration of the input audio.

text
string

The transcribed text.

words
array

Extracted words and their corresponding timestamps.


Show properties
segments
array

Segments of the transcribed text and their corresponding details.


Show properties
The transcription object (Verbose JSON)
{
  "task": "transcribe",
  "language": "english",
  "duration": 8.470000267028809,
  "text": "The beach was a popular spot on a hot summer day. People were swimming in the ocean, building sandcastles, and playing beach volleyball.",
  "segments": [
    {
      "id": 0,
      "seek": 0,
      "start": 0.0,
      "end": 3.319999933242798,
      "text": " The beach was a popular spot on a hot summer day.",
      "tokens": [
        50364, 440, 7534, 390, 257, 3743, 4008, 322, 257, 2368, 4266, 786, 13, 50530
      ],
      "temperature": 0.0,
      "avg_logprob": -0.2860786020755768,
      "compression_ratio": 1.2363636493682861,
      "no_speech_prob": 0.00985979475080967
    },
    ...
  ]
}
Chat
Given a list of messages comprising a conversation, the model will return a response.

Related guide: Chat Completions

Create chat completion
POST
 
https://api.openai.com/v1/chat/completions

Creates a model response for the given chat conversation.

Request body
messages
array

Required
A list of messages comprising the conversation so far. Example Python code.


Show possible types
model
string

Required
ID of the model to use. See the model endpoint compatibility table for details on which models work with the Chat API.

frequency_penalty
number or null

Optional
Defaults to 0
Number between -2.0 and 2.0. Positive values penalize new tokens based on their existing frequency in the text so far, decreasing the model's likelihood to repeat the same line verbatim.

See more information about frequency and presence penalties.

logit_bias
map

Optional
Defaults to null
Modify the likelihood of specified tokens appearing in the completion.

Accepts a JSON object that maps tokens (specified by their token ID in the tokenizer) to an associated bias value from -100 to 100. Mathematically, the bias is added to the logits generated by the model prior to sampling. The exact effect will vary per model, but values between -1 and 1 should decrease or increase likelihood of selection; values like -100 or 100 should result in a ban or exclusive selection of the relevant token.

logprobs
boolean or null

Optional
Defaults to false
Whether to return log probabilities of the output tokens or not. If true, returns the log probabilities of each output token returned in the content of message.

top_logprobs
integer or null

Optional
An integer between 0 and 20 specifying the number of most likely tokens to return at each token position, each with an associated log probability. logprobs must be set to true if this parameter is used.

max_tokens
integer or null

Optional
The maximum number of tokens that can be generated in the chat completion.

The total length of input tokens and generated tokens is limited by the model's context length. Example Python code for counting tokens.

n
integer or null

Optional
Defaults to 1
How many chat completion choices to generate for each input message. Note that you will be charged based on the number of generated tokens across all of the choices. Keep n as 1 to minimize costs.

presence_penalty
number or null

Optional
Defaults to 0
Number between -2.0 and 2.0. Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.

See more information about frequency and presence penalties.

response_format
object

Optional
An object specifying the format that the model must output. Compatible with GPT-4 Turbo and all GPT-3.5 Turbo models newer than gpt-3.5-turbo-1106.

Setting to { "type": "json_object" } enables JSON mode, which guarantees the message the model generates is valid JSON.

Important: when using JSON mode, you must also instruct the model to produce JSON yourself via a system or user message. Without this, the model may generate an unending stream of whitespace until the generation reaches the token limit, resulting in a long-running and seemingly "stuck" request. Also note that the message content may be partially cut off if finish_reason="length", which indicates the generation exceeded max_tokens or the conversation exceeded the max context length.


Show properties
seed
integer or null

Optional
This feature is in Beta. If specified, our system will make a best effort to sample deterministically, such that repeated requests with the same seed and parameters should return the same result. Determinism is not guaranteed, and you should refer to the system_fingerprint response parameter to monitor changes in the backend.

stop
string / array / null

Optional
Defaults to null
Up to 4 sequences where the API will stop generating further tokens.

stream
boolean or null

Optional
Defaults to false
If set, partial message deltas will be sent, like in ChatGPT. Tokens will be sent as data-only server-sent events as they become available, with the stream terminated by a data: [DONE] message. Example Python code.

stream_options
object or null

Optional
Defaults to null
Options for streaming response. Only set this when you set stream: true.


Show properties
temperature
number or null

Optional
Defaults to 1
What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.

We generally recommend altering this or top_p but not both.

top_p
number or null

Optional
Defaults to 1
An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.

We generally recommend altering this or temperature but not both.

tools
array

Optional
A list of tools the model may call. Currently, only functions are supported as a tool. Use this to provide a list of functions the model may generate JSON inputs for. A max of 128 functions are supported.


Show properties
tool_choice
string or object

Optional
Controls which (if any) tool is called by the model. none means the model will not call any tool and instead generates a message. auto means the model can pick between generating a message or calling one or more tools. required means the model must call one or more tools. Specifying a particular tool via {"type": "function", "function": {"name": "my_function"}} forces the model to call that tool.

none is the default when no tools are present. auto is the default if tools are present.


Show possible types
user
string

Optional
A unique identifier representing your end-user, which can help OpenAI to monitor and detect abuse. Learn more.

function_call
Deprecated
string or object

Optional
Deprecated in favor of tool_choice.

Controls which (if any) function is called by the model. none means the model will not call a function and instead generates a message. auto means the model can pick between generating a message or calling a function. Specifying a particular function via {"name": "my_function"} forces the model to call that function.

none is the default when no functions are present. auto is the default if functions are present.


Show possible types
functions
Deprecated
array

Optional
Deprecated in favor of tools.

A list of functions the model may generate JSON inputs for.


Show properties
Returns
Returns a chat completion object, or a streamed sequence of chat completion chunk objects if the request is streamed.

"""

In [3]:
len(aa)

18956

In [4]:
import lionagi as li

imodel = li.iModel("gpt-3.5-turbo")
compressor = LLMCompressor(imodel=imodel, n_samples=10, target_ratio=0.25)

In [5]:
bb = await compressor.compress(aa)

Original Token number: 4386
Selected Token number: 1024
Token Compression Ratio: 0.233
Compression Time: 1.840 seconds


In [6]:
branch = li.Branch()

res1 = await branch.chat(
    instruction="concisely explain what the text is about", 
    context=aa
)

branch2 = li.Branch()
res2 = await branch2.chat(
    instruction="concisely explain what the text is about", 
    context=bb
)

In [7]:
from IPython.display import display, Markdown

In [8]:
print(branch.messages[-1].metadata['extra']['usage'])
Markdown(res1)

{'prompt_tokens': 4845, 'completion_tokens': 97, 'total_tokens': 4942}


The text is a comprehensive guide on how to interact with the OpenAI API using HTTP requests. It covers installation instructions for official Python and Node.js libraries, authentication methods using API keys, and making API requests. The guide also explains how to handle responses, including streaming responses, and provides examples for various functionalities like generating chat completions, converting text to speech, transcribing audio, and translating audio. Additionally, it details the structure of request bodies and response objects for different API endpoints.

In [9]:
print(branch2.messages[-1].metadata['extra']['usage'])
Markdown(res2)

{'prompt_tokens': 1054, 'completion_tokens': 98, 'total_tokens': 1152}


The text provides detailed information on how to use OpenAI's API for generating and managing audio and chat completions. It covers supported audio formats, speed settings, and the process for making API requests, including the use of API keys for authentication. It also explains the importance of keeping API keys secure and the benefits of using project-specific keys over legacy user keys. Additionally, it describes how to stream responses, set various parameters such as temperature for randomness, and choose the format for transcript outputs.